In [30]:
import time
import random
import math
import psutil
import os
import tracemalloc
import csv
from datetime import datetime

In [31]:
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        left_half = arr[:mid]
        right_half = arr[mid:]

        merge_sort(left_half)
        merge_sort(right_half)

        i, j, k = 0, 0, 0
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                arr[k] = left_half[i]
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
            k += 1

        while i < len(left_half):
            arr[k] = left_half[i]
            i += 1
            k += 1

        while j < len(right_half):
            arr[k] = right_half[j]
            j += 1
            k += 1

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quick_sort(left) + middle + quick_sort(right)

def selection_sort(arr):
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]

def block_sort(arr):
    n = len(arr)
    if n <= 1:
        return arr

    block_size = int(math.sqrt(n))
    blocks = [[] for _ in range(block_size)]

    max_value = max(arr)
    min_value = min(arr)
    range_value = max_value - min_value

    for num in arr:
        index = (num - min_value) * block_size // (range_value + 1)
        blocks[index].append(num)

    print(f"Before sorting blocks: {blocks}")

    for block in blocks:
        block.sort()

    print(f"After sorting blocks: {blocks}")

    sorted_arr = []
    for block in blocks:
        sorted_arr.extend(block)

    print(f"Sorted array: {sorted_arr}")
    
    return sorted_arr

def measure_time_and_memory(arr, sort_function):
    tracemalloc.start()
    start_time = time.time()
    sort_function(arr)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return end_time - start_time, peak / (1024 * 1024)  # Convert to MB

def generate_best_case(n):
    return list(range(n))

def generate_worst_case(n):
    return list(range(n, 0, -1))

def generate_average_case(n):
    arr = list(range(n))
    random.shuffle(arr)
    return arr

def read_input_file(filename):
    with open(filename, 'r') as file:
        sizes = list(map(int, file.readline().strip().split()))
        algorithm = file.readline().strip()
    return sizes, algorithm

def get_sort_function(algorithm_name):
    if algorithm_name == "BlockSort":
        return block_sort
    else:
        raise ValueError(f"Algoritmo desconhecido: {algorithm_name}")

def main():
    sizes, algorithm_name = read_input_file('input.txt')
    sort_function = get_sort_function(algorithm_name)
    log_file = 'sort_performance_log.csv'

    if not os.path.exists(log_file):
        with open(log_file, 'w', newline='') as csvfile:
            fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    for size in sizes:
        cases = ['melhor', 'pior', 'medio']
        case_generators = [generate_best_case, generate_worst_case, generate_average_case]

        for case, gen_case in zip(cases, case_generators):
            if case == 'medio':
                times = []
                memories = []
                for _ in range(5):
                    arr = gen_case(size)
                    time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                    times.append(time_spent)
                    memories.append(memory_used)
                avg_time = sum(times) / len(times)
                avg_memory = sum(memories) / len(memories)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': avg_time,
                    'memory': avg_memory
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {avg_time:.6f}, {avg_memory:.6f} MB")
            else:
                arr = gen_case(size)
                time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': time_spent,
                    'memory': memory_used
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {time_spent:.6f}, {memory_used:.6f} MB")

            with open(log_file, 'a', newline='') as csvfile:
                fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(log_entry)

In [33]:
if __name__ == "__main__":
    main()

Before sorting blocks: [[0, 1, 2, 3], [4, 5, 6], [7, 8, 9]]
After sorting blocks: [[0, 1, 2, 3], [4, 5, 6], [7, 8, 9]]
Sorted array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2024-07-03 18:48:00, BlockSort, 10, melhor, 0.001001, 0.141870 MB
Before sorting blocks: [[4, 3, 2, 1], [7, 6, 5], [10, 9, 8]]
After sorting blocks: [[1, 2, 3, 4], [5, 6, 7], [8, 9, 10]]
Sorted array: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
2024-07-03 18:48:00, BlockSort, 10, pior, 0.000000, 0.000831 MB
Before sorting blocks: [[1, 2, 3, 0], [5, 4, 6], [7, 9, 8]]
After sorting blocks: [[0, 1, 2, 3], [4, 5, 6], [7, 8, 9]]
Sorted array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Before sorting blocks: [[0, 2, 1, 3], [5, 4, 6], [9, 8, 7]]
After sorting blocks: [[0, 1, 2, 3], [4, 5, 6], [7, 8, 9]]
Sorted array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Before sorting blocks: [[3, 1, 2, 0], [5, 6, 4], [8, 7, 9]]
After sorting blocks: [[0, 1, 2, 3], [4, 5, 6], [7, 8, 9]]
Sorted array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Before sorting blocks: [[1, 2, 0, 3], [4, 6, 5]

In [35]:
def block_sort(arr):
    n = len(arr)
    if n <= 1:
        return arr

    block_size = int(math.sqrt(n))
    blocks = [[] for _ in range(block_size)]

    max_value = max(arr)
    min_value = min(arr)
    range_value = max_value - min_value

    start_distribution = time.perf_counter()
    for num in arr:
        index = (num - min_value) * block_size // (range_value + 1)
        blocks[index].append(num)
    end_distribution = time.perf_counter()
    print(f"Time for distribution: {end_distribution - start_distribution:.10f} seconds")

    print(f"Before sorting blocks: {blocks}")

    start_sorting = time.perf_counter()
    for block in blocks:
        block.sort()
    end_sorting = time.perf_counter()
    print(f"Time for sorting blocks: {end_sorting - start_sorting:.10f} seconds")

    print(f"After sorting blocks: {blocks}")

    start_concatenation = time.perf_counter()
    sorted_arr = []
    for block in blocks:
        sorted_arr.extend(block)
    end_concatenation = time.perf_counter()
    print(f"Time for concatenation: {end_concatenation - start_concatenation:.10f} seconds")

    print(f"Sorted array: {sorted_arr}")
    
    return sorted_arr

def measure_time_and_memory(arr, sort_function):
    tracemalloc.start()
    start_time = time.perf_counter()
    sort_function(arr)
    end_time = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return end_time - start_time, peak / (1024 * 1024)  # Convert to MB

def generate_best_case(n):
    return list(range(n))

def generate_worst_case(n):
    return list(range(n, 0, -1))

def generate_average_case(n):
    arr = list(range(n))
    random.shuffle(arr)
    return arr

def read_input_file(filename):
    with open(filename, 'r') as file:
        sizes = list(map(int, file.readline().strip().split()))
        algorithm = file.readline().strip()
    return sizes, algorithm

def get_sort_function(algorithm_name):
    if algorithm_name == "BlockSort":
        return block_sort
    else:
        raise ValueError(f"Algoritmo desconhecido: {algorithm_name}")

def main():
    sizes, algorithm_name = read_input_file('input.txt')
    sort_function = get_sort_function(algorithm_name)
    log_file = 'sort_performance_log.csv'

    if not os.path.exists(log_file):
        with open(log_file, 'w', newline='') as csvfile:
            fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    for size in sizes:
        cases = ['melhor', 'pior', 'medio']
        case_generators = [generate_best_case, generate_worst_case, generate_average_case]

        for case, gen_case in zip(cases, case_generators):
            if case == 'medio':
                times = []
                memories = []
                for _ in range(5):
                    arr = gen_case(size)
                    time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                    times.append(time_spent)
                    memories.append(memory_used)
                avg_time = sum(times) / len(times)
                avg_memory = sum(memories) / len(memories)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': avg_time,
                    'memory': avg_memory
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {avg_time:.6f}, {avg_memory:.6f} MB")
            else:
                arr = gen_case(size)
                time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': time_spent,
                    'memory': memory_used
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {time_spent:.6f}, {memory_used:.6f} MB")

            with open(log_file, 'a', newline='') as csvfile:
                fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(log_entry)

if __name__ == "__main__":
    main()

Time for distribution: 0.0000130000 seconds
Before sorting blocks: [[0, 1, 2, 3], [4, 5, 6], [7, 8, 9]]
Time for sorting blocks: 0.0000051000 seconds
After sorting blocks: [[0, 1, 2, 3], [4, 5, 6], [7, 8, 9]]
Time for concatenation: 0.0000074000 seconds
Sorted array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2024-07-03 18:52:43, BlockSort, 10, melhor, 0.000513, 0.001277 MB
Time for distribution: 0.0000131000 seconds
Before sorting blocks: [[4, 3, 2, 1], [7, 6, 5], [10, 9, 8]]
Time for sorting blocks: 0.0000047000 seconds
After sorting blocks: [[1, 2, 3, 4], [5, 6, 7], [8, 9, 10]]
Time for concatenation: 0.0000094000 seconds
Sorted array: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
2024-07-03 18:52:43, BlockSort, 10, pior, 0.000554, 0.001219 MB
Time for distribution: 0.0000073000 seconds
Before sorting blocks: [[0, 2, 1, 3], [4, 6, 5], [8, 7, 9]]
Time for sorting blocks: 0.0000041000 seconds
After sorting blocks: [[0, 1, 2, 3], [4, 5, 6], [7, 8, 9]]
Time for concatenation: 0.0000092000 seconds
Sorted array: [